In [27]:
using Plots
using JLD2
using LaTeXStrings
using ColorTypes

In [28]:
include("parameters.jl")
using .MyParams

In [57]:
script_dir = @__DIR__
plot_folder = joinpath(script_dir, "plots")
if !isdir(plot_folder)
    mkdir(plot_folder)
end

In [58]:
dephasing = 100

100

In [59]:
data_dict = Dict()
for Ω in MyParams.Ω_values
    file_name = "results_data/Δ=$(MyParams.Δ),γ=$(MyParams.γ)/sz_mean_steady_for_$(MyParams.case)D,Ω=$(Ω),Δ=$(MyParams.Δ),γ=$(dephasing).jld2"
    data_dict[Ω] = load(file_name)
end

ArgumentError: ArgumentError: No file exists at given path: results_data/Δ=400,γ=100/sz_mean_steady_for_2D,Ω=3.5,Δ=400,γ=100.jld2

In [60]:
default(dpi=300)
p1 = plot(
    legend=false
)
for Ω in MyParams.Ω_values
    data = data_dict[Ω]
    t = data["t"]
    sz_mean_mean = vec(data["sz_mean_mean"])
    plot!(p1, t, sz_mean_mean, label="Ω=$(Ω)")
end
xlabel!(p1, L"t \Gamma")
ylabel!(p1, L"\langle s_z(t) \rangle")
title!(p1, "\\gamma = $(MyParams.γ), Δ = $(MyParams.Δ), $(MyParams.case)D, 
$(MyParams.nAtoms) atoms")
display(p1)
# savefig("plots/dynamic_rydberg_facilitation_$(MyParams.case)D,
$(MyParams.nAtoms)_atoms, γ = $(MyParams.γ), Δ = $(MyParams.Δ).png")

KeyError: KeyError: key 3.5 not found

Load data

Plots

In [61]:
selected_Ω = [0.1, 1:4..., 4.5, 5:0.2:7..., 7.3, 7.5, 8:0.5:20..., 21:2:59...]
p6 = plot(
    xscale=:log10,
    yscale=:log10,
    legend=false,
    xlim=(1e-0, 25),
    ylim=(1e-3, 1)
)
for Ω in MyParams.Ω_values
    Ω in selected_Ω || continue
    data = data_dict[Ω]
    plot!(p6, data["t"], vec(data["sz_mean_mean"]) .+ 1e-3, 
          label="Ω=$(Ω)", linewidth=3)
end
t_dash = data_dict[1]["t"]
dash_line = t_dash .^ (-MyParams.delta) .* 0.315
plot!(p6, t_dash, dash_line,
    linestyle=:dash, 
    color=:black, 
    linewidth=2, 
    label="t^(-$(MyParams.delta))"
)
xlabel!(p6, L"t \Gamma")
ylabel!(p6, L"\langle s_z(t) \rangle")
title!(p6, "\\gamma = $(MyParams.γ), Δ = $(MyParams.Δ), $(MyParams.case)D, 
$(MyParams.nAtoms) atoms")
display(p6)

KeyError: KeyError: key 4.0 not found

In [62]:
for Ω_crit in 8:0.1:9
    p5 = plot(
        legend=false,
        xlabel=L"t |\Omega - \Omega_{\text{c}}|^{\frac{\beta}{\delta}}",
        ylabel=L"t^\delta \langle s_z \rangle",
        title="Ω_crit = $Ω_crit, γ = $(MyParams.γ), Δ = $(MyParams.Δ), 
        $(MyParams.case)D, $(MyParams.nAtoms) atoms"
    )

    selected_Ω = [4.5, 5.0, 5.1, 5.3, 5.5, 5.7, 9.5, 10.0, 11.0]
    
    for Ω in selected_Ω
        data = data_dict[Ω]
        t = data["t"]
        sz_mean_mean = vec(data["sz_mean_mean"])
        
        # Filter valid data points
        valid_data = findall(t .> 0 .&& sz_mean_mean .> 0)
        isempty(valid_data) && continue
        
        # Calculate scaled coordinates
        x_scaled = t[valid_data] .* abs.(Ω - Ω_crit) .^ (MyParams.beta / MyParams.delta)
        y_scaled = t[valid_data] .^ MyParams.delta .* sz_mean_mean[valid_data]
        
        # Plot if we have valid points
        plot!(p5, log.(x_scaled), log.(y_scaled), linewidth=2)
    end
    display(p5)
end

KeyError: KeyError: key 4.5 not found

Omega vs. sz

In [63]:
p2 = plot(
    title="γ = $(MyParams.γ), Δ = $(MyParams.Δ), $(MyParams.nAtoms) atoms",
    xlabel=L"\Omega",
    ylabel=L"\langle s_z \rangle"
)

# Define selected Ω values more concisely
selected_Ω = [
    0.1, 1:4..., 4.5, 5.0, 5.3, 5.7,
    6.1:0.2:8.0..., 8.5,
    9.0:0.5:25.0...,
    27:2:59...
]

final_sz_values = Float64[]
Ωvalues = Float64[]
Ω_crit = 9

for Ω in MyParams.Ω_values
    Ω in selected_Ω || continue
    data = data_dict[Ω]
    sz_mean_mean = vec(data["sz_mean_mean"])
    push!(final_sz_values, last(sz_mean_mean))
    push!(Ωvalues, Ω)
end

scatter!(
    Ωvalues, final_sz_values,
    marker=:circle, markersize=6, color=:blue,
    linestyle=:auto, legend=:topright, label=false
)

vline!([Ω_crit], color=:black, linestyle=:dash, linewidth=2, label=false)
display(p2)

KeyError: KeyError: key 4.0 not found

ln|Ω_values .- Ω_crit| vs. ln(<'Sz'>)

In [64]:
Ω_crit_values = 8:0.05:8.5
fixed_beta = MyParams.beta
prefactor = 0.069
for Ω_crit in Ω_crit_values
    final_sz_values = Float64[]
    filtered_Ω_values = Float64[]
    for Ω in MyParams.Ω_values
        if Ω - Ω_crit > 0.1
            data = data_dict[Ω]
            sz_mean_mean = vec(data["sz_mean_mean"])
            last_value = last(sz_mean_mean)
            push!(final_sz_values, last_value)
            push!(filtered_Ω_values, Ω)
        end
    end
    xaxis = log.(abs.(filtered_Ω_values .- Ω_crit))
    yaxis = log.(final_sz_values)
    p3 = scatter(
        xaxis, 
        yaxis,
        marker=:circle,
        markersize=6,
        color=:blue,
        legend=:topleft,
        label="sz",
        #xlim=[-0.5,1.25],
        #ylim=[-2, -0.9],
        title= "Ω_crit = $Ω_crit, γ = $(MyParams.γ), Δ = $(MyParams.Δ), $(MyParams.case)D, 
        $(MyParams.nAtoms) atoms",
        xlabel=L"\ln|\Omega - \Omega_\mathrm{crit}|",
        ylabel=L"\ln(\langle s_z \rangle)",
    )
    power_law_y = log.(abs.(filtered_Ω_values .- Ω_crit) .^ fixed_beta .* prefactor)
    plot!(
        xaxis, 
        power_law_y,
        color=:red,
        linestyle=:dash,
        label="|Ω - Ω_crit|^$(fixed_beta)",
        linewidth=3
    )
    display(p3)
end

KeyError: KeyError: key 8.5 not found

In [65]:
# nAtoms_values = [100,225,400]
# Ω = 6.2

In [66]:
# data_dict = Dict()
# for natoms in nAtoms_values
#     file_name = "results_data/sz_mean_steady_for_$(MyParams.case)D, $(natoms)_atoms, Ω=$(Ω).jld2"
#     data_dict[natoms] = load(file_name)
# end

In [67]:
# p10 = plot(
#     xscale=:log10,
#     yscale=:log10,
#     legend=:bottomleft,
#     xlim=[2,25],
#     #ylim=[0.01,0.4],
# )
# for natoms in nAtoms_values
#     data = data_dict[natoms]
#     t = data["t"]
#     sz_mean_mean = vec(data["sz_mean_mean"])
#     plot!(p10, t .+ 1e-3, sz_mean_mean, label="$(natoms) atoms,Ω=$(Ω), Δ = $(MyParams.Δ)", linewidth=3)
# end
# data = data_dict[100]
# t = data["t"]
# t_dash = t
# dash_line = t_dash .^ (-MyParams.delta) .* 0.315
# plot!(p10, t_dash, dash_line,
#     linestyle=:dash, 
#     color=:black, 
#     linewidth=2, label="t^(-$(MyParams.delta))")
# xlabel!(p10, L"t \Gamma")
# ylabel!(p10, L"\langle s_z(t) \rangle")
# display(p10)